- *bar*:
  - h/v
  - line width
- *line*:
  - simple/patterned
  - point-to-point/ray-to-infinity/ray-to-hit
  - line-width
- *dot* = square of size 1
- *square* = rectangle of equal width
- *rectangle*:
  - fill-color
  - line-width
- *paste*

In [17]:
import enum
import typing
import itertools

import attrs
import numpy as np

In [23]:
Axis4 = enum.StrEnum("Axis", [(k,k) for k in "vertical horizontal".split()])
Axis8 = enum.StrEnum("Axis", [(k,k) for k in "vertical horizontal".split()])
Dir4 = enum.StrEnum("Dir4", [(k,k) for k in "right up left down".split()])
Dir8 = enum.StrEnum("Dir8", [(k,k) for k in "right up_right up up_left left down_left down down_right".split()])

class Colour(enum.StrEnum):
    BLACK = "#000000"
    BLUE = "#0074D9"
    RED = "#FF4136"
    GREEN = "#2ECC40"
    YELLOW = "#FFDC00"
    GRAY = "#AAAAAA"
    MAGENTA = "#F012BE"
    ORANGE = "#FF851B"
    CYAN = "#7FDBFF"
    BROWN = "#870C25"

Paint = enum.StrEnum("Paint", [("NONE","")] + [(c.name,c.value) for c in Colour])

for e in [Axis, Dir8, Dir4, Paint, Colour]:
    for v in e:
        globals()[v.name] = v

H = EW
V = NS

T = typing.TypeVar("T")

@attrs.frozen
class _Logical:
    x: int # indexes E-W
    y: int # indexes S-N

@attrs.frozen
class Delta(_Logical):
    pass

@attrs.frozen
class _Physical:
    x: int
    y: int


@attrs.frozen
class Size(_Physical):
    pass

@attrs.frozen
class Pos(_Physical):
    @classmethod
    def origin(cls) -> typing.Self:
        """ *logical* origin. """
        ctxt = Context.current
        s = ctxt.canvas_size
        v = ctxt.orienation.inverse.value
        return cls(s.i-1 if v&0b010 else 0, s.j-1 if v&0b001 else 0)


@attrs.frozen
class Canvas:
    data: np.ndarray # indexed by [i,j]; maps to Paint index

    def count_cells(self) -> int:
        return np.sum(self.data>0)

    def psum(self) -> PaintCount:
        return PaintCount(np.bincount(self.data.ravel(),minlength=len(Paint)))

@attrs.frozen
class Mask:
    data: np.ndarray # indexed by [i,j]; maps to boolean

@attrs.frozen
class PaintCount:
    data: np.ndarray # indexed by [paint]; int

    def maxp(self) -> tuple[Paint,int]:
        d = self.data
        i = np.argmax(d)
        return Paint[i],d[i]

    def maxc(self) -> tuple[Colour,int]:
        d = self.data[1:]
        i = np.argmax(d)
        return Colour[i],d[i]


@attrs.frozen
class _Projected:
    a: Axis  # the axis that is *not* projected

@attrs.frozen
class PPos(_Projected):
    p: int

@attrs.frozen
class PCanvas(_Projected):
    data: np.ndarray # indexed by [pos,paint]; colour sum

    def maxp(self) -> tuple[PPos, Paint, int]:
        d = self.data
        p,q = np.unravel_index(np.argmax(d),d.shape)
        return PPos(a=self.a,p=p),Paint[q],d[p,q]

    def maxc(self) -> tuple[PPos, Colour, int]:
        d = self.data[:,1:]
        p,q = np.unravel_index(np.argmax(d),d.shape)
        return PPos(a=self.a,p=p),Colour[q],d[p,q]


@attrs.frozen
class PArray(_Projected):
    data: np.ndarray # indexed by [pos]; boolean

@attrs.frozen
class PCount(_Projected):
    data: np.ndarray # indexed by [pos]; int





In [19]:
class SymOp(enum.Enum):
    e = 0b000
    x = 0b001 # flip the x-coordinates
    y = 0b010 # flip the y-coordinates
    i = 0b011
    t = 0b100
    r = 0b101 # clock-wise rotation when x->right, y->down (vision coordinate system)
    l = 0b110
    d = 0b111

    @property
    def inverse(self):
        S = type(self)
        return {S.l: S.r, S.r: S.l}.get(self, self)

    def combine(self, rhs):
        """ Apply `rhs` first, then `self`
        """
        v = rhs.value
        s = self.value
        if s & 0b100 and bool(v & 0b001) != bool(v & 0b010):
            # we need to swap flip_x and flip_y
            v ^= 0b011
        v ^= s
        return type(self)(v)

    def apply_spacial(self, image):
        s = self
        transpose = bool(s.value & 0b100)
        flip_y = bool(s.value & 0b010)
        flip_x = bool(s.value & 0b001)
        img = image
        if transpose:
            img = img.transpose(1, 0, *range(2,img.ndim))
        if flip_y:
            img = img[::-1, :]
        if flip_x:
            img = img[:, ::-1]
        return img


@attrs.frozen
class Context:
    canvas_size: Size
    foreground: Paint = Paint.none
    background: Paint = Paint.none
    monochrome: bool = False
    # maps logical to physical:
    orientation: SymOp = SymOp.e 

    @property
    @classmethod
    def current(cls)->typing.Self:
        pass

In [ ]:
def solve_1ae2feb7(canvas: Canvas) -> Canvas:
    # background handling
    bg = background(canvas)
    canvas = exclude(canvas, match_color(canvas, bg))
    # find vertical divider
    divider_col = argmax(most_common_color_cell_count(col, exclude=bg) for col in iter_cols(canvas))
    # identify where the spec is
    left,right = split_cols(canvas, divider_col)
    if cell_count(left,exclude=bg) > cell_count(right, exclude=bg):
        print("This one is reversed")
        # flip logical x coordinates (E/W) - positions don't move on the canvase
        Context.apply(transform=SymOp.x)
        spec = hi
    else:
        spec = lo
    out = inp.clone()
    for y,s in spec.enumerate(V):
        for paint, group in itertools.groupby(s.enumerate(), key=lambda xp:xp[1]
        